In [ ]:
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/hex_maps/"
shots_DIR = "../data/ShotLocationData/"

In [ ]:
year = 2023
shots = pd.read_parquet(shots_DIR + f"NBA_Shot_Details_{year}.parquet")
shots_ps = pd.read_parquet(shots_DIR + f"NBA_Shot_Details_{year}_PS.parquet")

In [ ]:
player_shotchart = ShotChartDetail(
    league_id="00",
    team_id=0,
    player_id=2544,
    context_measure_simple="FGA",
    season_nullable=get_ss(year),
    season_type_all_star="Regular Season"
)  # , season_nullable=season)
league_avg = player_shotchart.get_data_frames()[1]
player_shotchart = ShotChartDetail(
    league_id="00",
    team_id=0,
    player_id=2544,
    context_measure_simple="FGA",
    season_nullable=get_ss(year),
    season_type_all_star="Playoffs"
)  # , season_nullable=season)
league_avg_ps = player_shotchart.get_data_frames()[1]

In [ ]:
bin_no = 50.0 / 1
binned_df = create_bins(
    data_frame=shots,
    league_average=league_avg,
    bin_number_x=bin_no,
    bin_number_y=300 / (500.0 / bin_no),
)
data_frame = binned_df
dropped_dups = data_frame.drop_duplicates(
    subset=["BIN_LOC_X", "BIN_LOC_Y"], keep="first"
)

In [ ]:
bin_no = 50.0 / 1
binned_df_ps = create_bins(
    data_frame=shots_ps,
    league_average=league_avg_ps,
    bin_number_x=bin_no,
    bin_number_y=300 / (500.0 / bin_no),
)
data_frame_ps = binned_df_ps
dropped_dups_ps = data_frame_ps.drop_duplicates(
    subset=["BIN_LOC_X", "BIN_LOC_Y"], keep="first"
)

In [ ]:
df1 = dropped_dups
df2 = dropped_dups_ps
df1 = df1[df1["SHOT_TYPE"] == "3PT Field Goal"]
# df1 = df1.query("SHOT_ZONE_BASIC == 'Mid-Range'")
# df1 = df1.nlargest(100,"LOC_RAW_COUNTS")
df2 = df2.nlargest(300,"LOC_RAW_COUNTS")

In [ ]:
len(df1)

In [ ]:
df2.columns

In [ ]:
bball_gray = "#312f30"
bball_white = "#dddee0"
bball_orange = "#f87c24"
bball_light_orange = "#fbaf7b"
bball_black = "#000010"
dark_grey = "#282828"

colorscale = "Plasma"

fig = go.Figure()

draw_plotly_court(fig, lw=1, margins=13, lcolor=bball_orange)
fig.add_trace(
    go.Scatter(
        x=df1.BIN_LOC_X,
        y=df1.BIN_LOC_Y,
        mode="markers",
        name="markers",
        marker=dict(
            # size=dropped_dups.LOC_COUNTS,
            size=10,
            color="red",
            # line=dict(width=0.5, color="White"),
            symbol="square-dot",
        ),
    )
)
fig.update_layout(
    plot_bgcolor=bball_black,
    paper_bgcolor=bball_black,
    title=dict(
        text="Top 100 Shot Locations: 3 Point",
        y=0.975,
        x=0.06,
        xanchor="auto",
        yanchor="middle",
    ),
    font=dict(family="Arial, Tahoma, Helvetica", size=15, color="Orange"),
)
fig.write_image(f"Midrange Shot Locations {year+1}.png", scale=5)
fig.show()

In [ ]:
bball_gray = "#312f30"
bball_white = "#dddee0"
bball_orange = "#f87c24"
bball_light_orange = "#fbaf7b"
bball_black = "#000010"
dark_grey = "#282828"

colorscale = "Plasma"

fig = go.Figure()

draw_plotly_court(fig, lw=1, margins=13, lcolor=bball_orange)
fig.add_trace(
    go.Scatter(
        x=df2.BIN_LOC_X,
        y=df2.BIN_LOC_Y,
        mode="markers",
        name="markers",
        marker=dict(
            # size=dropped_dups.LOC_COUNTS,
            size=10,
            color="red",
            # line=dict(width=0.5, color="White"),
            symbol="square-dot",
        ),
    )
)
fig.update_layout(
    plot_bgcolor=bball_black,
    paper_bgcolor=bball_black,
    title=dict(
        text="Top 300 Shot Locations: Playoffs 2022-23",
        y=0.975,
        x=0.06,
        xanchor="auto",
        yanchor="middle",
    ),
    font=dict(family="Arial, Tahoma, Helvetica", size=15, color="Orange"),
)
fig.write_image(f"Playoffs Shot Locations {year+1}.png", scale=5)
fig.show()